# Catboost Lv1 Hyperparameter Sweep 20211103
Largely after https://www.kaggle.com/satorushibata/optimize-catboost-hyperparameter-with-optuna-gpu

# Setup

In [1]:
# two manual flags (ex-config)
COLAB = False
USE_GPU = True

In [2]:
# basic imports
from pathlib import Path
import os
import math
from datetime import datetime
import random
import multiprocessing
import pickle

In [3]:
%matplotlib inline
%config Completer.use_jedi = False
os.environ['WANDB_NOTEBOOK_NAME'] = f"sweep_xgboost_{datetime.now().strftime('%Y%m%d')}.ipynb"

In [4]:
# handle Google Colab-specific library installation/updating
if COLAB:
    # much of the below inspired by or cribbed from the May 2021 Kaggle Tabular Playground winner, at 
    # https://colab.research.google.com/gist/academicsuspect/0aac7bd6e506f5f70295bfc9a3dc2250/tabular-may-baseline.ipynb?authuser=1#scrollTo=LJoVKJb5wN0L
    
    # Kaggle API for downloading the datasets
    !pip install --upgrade -q kaggle

    # weights and biases
    !pip install -qqqU wandb
    
    # Optuna for parameter search
    !pip install -q optuna

    # !pip install --upgrade xgboost

    # upgrade sklearn
    !pip install --upgrade scikit-learn

    # !pip install category_encoders
    # !pip install catboost
#     !pip install --upgrade -q lightgbm

    # lighgbm gpu compatible
    # !git clone --recursive https://github.com/Microsoft/LightGBM
    # ! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;
    
    # # this part is from https://github.com/rapidsai/gputreeshap/issues/24
    # !pip install cmake --upgrade
    # # !pip install sklearn --upgrade
    # !git clone --recursive https://github.com/dmlc/xgboost
    # %cd /content/xgboost
    # !mkdir build
    # %cd build
    # !cmake .. -DUSE_CUDA=ON
    # !make -j4
    # %cd /content/xgboost/python-package
    # !python setup.py install --use-cuda --use-nccl
    # !/opt/bin/nvidia-smi
    # !pip install shap
    

Now, non-stdlib imports

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# general ML tooling
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import log_loss, roc_auc_score
import wandb
from optuna.integration.wandb import WeightsAndBiasesCallback
# from wandb.xgboost import wandb_callback
# from wandb.lightgbm import wandb_callback
# from sklearn.impute import KNNImputer, StandardImputer
# import timm

import seaborn as sns
# import xgboost
from catboost import CatBoostClassifier
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
# from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import RobustScaler
# from sklearn.feature_selection import SelectKBest, f_regression
from joblib import dump, load
# feature engineering tools
# from sklearn.feature_selection import mutual_info_regression
# import featuretools as ft


from optuna.samplers import TPESampler
import optuna
# import catboost
from sklearn.utils import resample
import sklearn.metrics

Now, datapath setup

In [6]:
# # This is the code for reading the train.csv and converting it to a .feather file
# df = pd.read_csv(datapath/'train.csv', index_col='id', low_memory=False)
# df.index.name = None
# df.to_feather(path='./dataset_df.feather')

In [7]:
if COLAB:
    # mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    
    # handling datapath
    datapath = Path('/content/drive/MyDrive/kaggle/tabular_playgrounds/nov2021/')
    
else:
    # if on local machine
#     datapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/sep2021/')  
    root = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/nov2021/')
    datapath = root/'datasets'
    # edapath = root/'EDA'
    # modelpath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/models/')
    predpath = root/'preds'
    subpath = root/'submissions'
    studypath = root/'studies'
    
    for pth in [datapath, predpath, subpath, studypath]:
        pth.mkdir(exist_ok=True)
    


In [8]:

# n_trials = int(1000)
SEED = 42

In [9]:
# Function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
seed_everything(SEED)

## Ex-Model Config

In [10]:
# meta-config for preprocessing and cross-validation, but NOT for model parameters
# in the sweep version, this includes both ex-model parameters and defaults for model parameters
exmodel_config = {
    # model config
    "arch": 'catboost',
    "type": 'sweep',
    "level": 1,
#     "model": XGBClassifier,
#     "n_estimators": 100, 
#     "max_depth": 3,
#     "learning_rate": 0.1,
#     "test_size": 0.2,
#     "reg_lambda": None, 
#     "scaler": "sklearn.preprocessing.StandardScaler()", # TODO: experiment with others (but imputation may be slow)
#     "scale_b4_impute": False,
#     "imputer": "sklearn.impute.SimpleImputer(strategy='median', add_indicator=True)",
#     "knn_imputer_n_neighbors": None, # None if a different imputer is used
#     "feature_selector": SelectKBest,
#     "k_best": 80,
#     "feature_selection_scoring": f_regression,
    'random_state': SEED,
    'tuner': "Optuna",
#     'optuna_trials': 20,
#     'subsample': 1,
    'cross_val_strategy': KFold, # None for holdout, or the relevant sklearn class
    'kfolds': 5, # if 1, that means just doing holdout
    'test_size': 0.2,
    # these are XGBoost default (my choice) params 
#     "tree_method": "auto", # set to 'gpu_hist' to try GPU if available
#     "booster": 'gbtree', # dart may be marginally better, but will opt for this quicker approach as a default
#     "n_estimators": 200, 
#     "max_depth": 3,
#     "learning_rate": 0.1,
#     "n_jobs": -1,
#     "verbosity": 1,
#     "subsample": 1,
#     'features_created': False,
#     'feature_creator': None,
}


## Data Setup

**TODO** Write some conditional logic here to automate it -- possibly as part of a sklearn.*pipeline

In [11]:
# train_source = datapath/'train.feather'
# df = pd.read_feather(path=datapath/'train.feather')
# y = np.array(df.target)
# dump(y, filename=datapath/'y.joblib')
# del df

y = load(datapath/'y_orig.joblib')

# df.index.name = 'id'
# y_train = df.target
# features = [x for x in df.columns if x != 'target']
# X_train = df[features]
# # X.index.name = 'id'
# # y.index.name = 'id'
# X = np.array(X_train)
# y = np.array(y_train)

# del df, X_train, y_train

# load the Boruta-filtered green-zone 98 features (based on 200 iterations of the algo)
# train_source = '/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/alt_datasets/X_boruta_200iter_filtered_green.joblib'
# X = load(train_source)

train_source = datapath/'X_orig.feather'
X = pd.read_feather(train_source)

# exmodel_config['feature_count'] = len(X.columns)
exmodel_config['feature_count'] = X.shape[1]
exmodel_config['instance_count'] = X.shape[0]
exmodel_config['scaler'] = str(RobustScaler())
# exmodel_config['feature_generator'] = None
# exmodel_config['feature_generator'] = "Summary statistics"

exmodel_config['train_source'] = str(train_source)
# test_source = datapath/'test.feather'
# exmodel_config['test_source'] = str(test_source)
# X_test = pd.read_feather(path=test_source)
# X_test = X_test.iloc[:, 1:]
# X_test = np.array(X_test)

In [12]:
wandb_kwargs = {
    # wandb config
    'name': f"{os.environ['WANDB_NOTEBOOK_NAME'][:-6]}_{datetime.now().strftime('%H%M%S')}", # just removes the .ipynb extension, leaving the notebook filename's stem
    'project': '202111_Kaggle_tabular_playground',
    'tags': ['sweep'],
    'notes': f"Sweep for {exmodel_config['arch']} on GPU with {exmodel_config['tuner']}, using original dataset with {exmodel_config['scaler']}",
    'config': exmodel_config,
}

In [13]:
X.shape, y.shape

((600000, 100), (600000,))

In [14]:
type(X), type(y)

(pandas.core.frame.DataFrame, pandas.core.series.Series)

# Experiment setup

In [15]:
# wandb_kwargs = {
#     # wandb config:
#     'name': f"{os.environ['WANDB_NOTEBOOK_NAME'][:-6]}_{datetime.now().strftime('%H%M%S')}", # just removes the .ipynb extension, leaving the notebook filename's stem
#     'project': '202109_Kaggle_tabular_playground',
#     'tags': ['sweep'],
#     'notes': "Sweep for CatBoost using Optuna",
#     'config': exmodel_config,
# }

In [20]:
# originally from https://www.kaggle.com/satorushibata/optimize-catboost-hyperparameter-with-optuna-gpu
def objective(trial, arch=exmodel_config['arch']):
    # split the (original Kaggle training) data into partitions
    # if study.best_trial:
    #     print("Dumping best params, which are:")
    #     print(str(study.best_trial.params))
    #     dump(study.best_trial.params, filename=datapath/'optuna_catboost_best_20210920.joblib')
       
    # else:
    #     print("No best study yet")
    # X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=int(SEED), shuffle=True)
    # # create wrappers for the training and validation partitions
    # train_pool = catboost.Pool(X_train, y_train)
    # valid_pool = catboost.Pool(X_valid, y_valid)
    
    # experimental parameters
    params = {
        'iterations' : trial.suggest_int('iterations', 2000, 30000),                         
        'depth' : trial.suggest_int('depth', 3, 10),                                       
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 0.5),               
        'random_strength': trial.suggest_int('random_strength', 0, 100), 
#         'objective': trial.suggest_categorical('objective', ['Logloss', 'CrossEntropy']),
#         'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['MVC', 'Bernoulli']),#, 'Poisson']),
        'od_wait': trial.suggest_int('od_wait', 20, 2000),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 2, 70), # aka l2_leaf_reg
        'border_count': trial.suggest_int('border_count', 50, 275),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 20), # aka min_data_in_leaf
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations', 1, 5),
        # 'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
#         'subsample': trial.suggest_uniform('subsample', 0.5, 1),
#         'learning_rate' :trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
        # 'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
#         'max_leaves': trial.suggest_int('max_leaves', 32, 128)
    }
    device = "GPU" #if USE_GPU else "CPU"
    # instantiate the model, with some parameters locked in, and experimnental ones passed via splat 
    model = CatBoostClassifier(
        # loss_function="Logloss",
        eval_metric="AUC",
        task_type=device,
        objective='Logloss',
#         l2_leaf_reg=50,
        random_seed=SEED,
        verbose=False,
#         border_count=64,
        **params
    )        
    
    kfold = exmodel_config['cross_val_strategy'](n_splits=exmodel_config['kfolds'], shuffle=True, random_state=SEED)
    # initialize lists for out-of-fold preds and ground truth
    oof_preds, oof_y = [], []
    for fold, (train_ids, valid_ids) in enumerate(kfold.split(X,y)):
        print(f"FOLD {fold}")
        print("---------------------------------------------------")
        y_train, y_valid = y[train_ids], y[valid_ids] # y will be an np.ndarray already; handling will be same regardless of model
        if isinstance(X, np.ndarray):
            X_train, X_valid = X[train_ids], X[valid_ids]
        else:
            X_train, X_valid = X.iloc[train_ids,:], X.iloc[valid_ids,:] # bc need pandas.DataFrames for ce
        scaler = RobustScaler()
        X_train = scaler.fit_transform(X_train)
        X_valid = scaler.transform(X_valid)

        model.fit(X_train, y_train)
        y_valid_preds = model.predict_proba(X_valid)[:,1]

        # add the fold-model's OOF preds and ground truths to the out-of-loop lists
        oof_preds.extend(y_valid_preds)
        oof_y.extend(y_valid)
        fold_roc_auc = roc_auc_score(y_score=y_valid_preds, y_true=y_valid)
        print(f"Fold {fold} ROC AUC is: {fold_roc_auc}")
    
    model_roc_auc = roc_auc_score(y_true=oof_y, y_score=oof_preds)
    print(f"Valid AUC score for {arch} model is {model_roc_auc}")

    return model_roc_auc

In [17]:
wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)

<ipython-input-17-69ea9289a2cf>:1: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.
  wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find sweep_xgboost_20211103.ipynb
wandb: Currently logged in as: hushifang (use `wandb login --relogin` to force relogin)


In [18]:
study = optuna.create_study(direction = "maximize", 
                            sampler = TPESampler(seed=int(SEED)), 
                            study_name=f"{exmodel_config['arch']}-lv{exmodel_config['level']}-{exmodel_config['scaler']}-orig_dataset-{datetime.now().strftime('%Y%m%d')}")
# setting n_jobs=1 bc wandb apparently requires this


[I 2021-11-03 15:48:08,848] A new study created in memory with name: catboost-lv1-RobustScaler()-orig_dataset-20211103


In [21]:
for x in range(100):
    study.optimize(objective, n_trials = 1, callbacks = [wandbc]) #n_jobs = multiprocessing.cpu_count())
    dump(study, filename=studypath/f"optuna-{exmodel_config['arch']}-lv{exmodel_config['level']}-study-trial{x}-{datetime.now().strftime('%Y%m%d')}.joblib")

FOLD 0
---------------------------------------------------
Fold 0 ROC AUC is: 0.7074549952712735
FOLD 1
---------------------------------------------------
Fold 1 ROC AUC is: 0.7070464024423961
FOLD 2
---------------------------------------------------
Fold 2 ROC AUC is: 0.70505314885779
FOLD 3
---------------------------------------------------
Fold 3 ROC AUC is: 0.7054408738500189
FOLD 4
---------------------------------------------------
Fold 4 ROC AUC is: 0.7043855755526114


[I 2021-11-03 18:02:22,965] Trial 1 finished with value: 0.7058742939670087 and parameters: {'iterations': 21826, 'depth': 3, 'learning_rate': 0.41472250004816347, 'random_strength': 84, 'od_wait': 440, 'reg_lambda': 14.364097770082843, 'border_count': 91, 'min_child_samples': 7, 'leaf_estimation_iterations': 3}. Best is trial 1 with value: 0.7058742939670087.


Valid AUC score for catboost model is 0.7058742939670087
FOLD 0
---------------------------------------------------
Fold 0 ROC AUC is: 0.7483466209615757
FOLD 1
---------------------------------------------------
Fold 1 ROC AUC is: 0.7488390878281969
FOLD 2
---------------------------------------------------
Fold 2 ROC AUC is: 0.7467021911705791
FOLD 3
---------------------------------------------------
Fold 3 ROC AUC is: 0.7472263659643393
FOLD 4
---------------------------------------------------
Fold 4 ROC AUC is: 0.7449946013011932


[I 2021-11-03 18:11:34,169] Trial 2 finished with value: 0.7472172184981711 and parameters: {'iterations': 14094, 'depth': 5, 'learning_rate': 0.044809759182149515, 'random_strength': 14, 'od_wait': 598, 'reg_lambda': 26.912605343971038, 'border_count': 153, 'min_child_samples': 16, 'leaf_estimation_iterations': 1}. Best is trial 2 with value: 0.7472172184981711.


Valid AUC score for catboost model is 0.7472172184981711
FOLD 0
---------------------------------------------------
Fold 0 ROC AUC is: 0.7437772816440829
FOLD 1
---------------------------------------------------
Fold 1 ROC AUC is: 0.744011041448424
FOLD 2
---------------------------------------------------
Fold 2 ROC AUC is: 0.7416178554437498
FOLD 3
---------------------------------------------------
Fold 3 ROC AUC is: 0.7420972182649239
FOLD 4
---------------------------------------------------
Fold 4 ROC AUC is: 0.7396862067672051


...

[I 2021-11-04 09:44:19,891] Trial 44 finished with value: 0.7493188570215233 and parameters: {'iterations': 28722, 'depth': 7, 'learning_rate': 0.007583819650520586, 'random_strength': 0, 'od_wait': 1633, 'reg_lambda': 51.161973778356966, 'border_count': 221, 'min_child_samples': 12, 'leaf_estimation_iterations': 2}. Best is trial 22 with value: 0.7493576264007167.


Valid AUC score for catboost model is 0.7493188570215233
FOLD 0
---------------------------------------------------
Fold 0 ROC AUC is: 0.7497276703828812
FOLD 1
---------------------------------------------------
Fold 1 ROC AUC is: 0.7506919854438707
FOLD 2
---------------------------------------------------
Fold 2 ROC AUC is: 0.74822134759519
FOLD 3
---------------------------------------------------
Fold 3 ROC AUC is: 0.74835228724934
FOLD 4
---------------------------------------------------
Fold 4 ROC AUC is: 0.7465664035341656


[I 2021-11-04 10:10:08,606] Trial 45 finished with value: 0.7487070557060025 and parameters: {'iterations': 29820, 'depth': 6, 'learning_rate': 0.014518986184677889, 'random_strength': 14, 'od_wait': 1783, 'reg_lambda': 58.708815060203484, 'border_count': 268, 'min_child_samples': 15, 'leaf_estimation_iterations': 2}. Best is trial 22 with value: 0.7493576264007167.


Valid AUC score for catboost model is 0.7487070557060025
FOLD 0
---------------------------------------------------


KeyboardInterrupt: 

In [22]:
dump(study, filename=studypath/f"optuna-{exmodel_config['arch']}_lv{exmodel_config['level']}-thru_trial_45-{datetime.now().strftime('%Y%m%d')}.joblib")

['/media/sf/easystore/kaggle_data/tabular_playgrounds/nov2021/studies/optuna-catboost_lv1-thru_trial_45-20211104.joblib']

In [23]:
study.best_trial.params

{'iterations': 29222,
 'depth': 7,
 'learning_rate': 0.0067277390824230605,
 'random_strength': 1,
 'od_wait': 1989,
 'reg_lambda': 51.436909447809484,
 'border_count': 239,
 'min_child_samples': 11,
 'leaf_estimation_iterations': 2}

In [24]:
wandb.log({'catboost_params': study.best_trial.params})
wandb.finish()

border_count,▂▄█▁▆▄█▄▅██▇▇▇▆▇▇▆▇▇▇█▅▆▇▇▂▆▇▇█▆▄▃▇▆▁▇▇█
depth,▁▃▅▁▃█▃▇▆▄▄▅▅▆▄▆▅▇▄▅▅▄▆▇▅▄▂▆▅▅▅▄▅▆█▄▃▅▅▄
iterations,▆▄▅▃▆▅▃▁▂████▇▇▇█▆▇██▇█▇▆▇▆▂██▇▇▇▄▄▆▆███
leaf_estimation_iterations,▅▁█▃██▁█▁▃▃▃▃▆▃▅▃▆▁▃▃▅▃▁▅▅▃▁▃▅▃▁▅▃▁▃▁▃▃▃
learning_rate,█▂▁▂▁▁▁▂▃▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▂▁▁▁▁▁
min_child_samples,▃▇███▃▇▂▄▅▅▆▅▆▄▆▄▂▁▅▅▆▇▄▆▄▃▅▅▅▇▆▄▅▃▇▇▅▅▆
od_wait,▂▃▂▂▂▁▇▆▆████▅▅▇▆▄▆█▇▇█▇▇▄▇▅██▇█▆█▇▁▃██▇
random_strength,▇▂▅▄▅▂▂▆▅▁▁▁▁█▄▃▃▁▄▁▂▁▂▃▁▁▇▂▁▂▁▂▃▂▄▁▂▁▁▂
reg_lambda,▂▃▁▄█▃▁▁▄▆▆▆▆▇▅▇▅▇█▆▅▇▅▄▇▇▂▅▆▆▆▇█▅▆▇▆▆▆▇
value,▂█▇████▇▇█████████▃██████▇█▇██▁█▇█▇█████
border_count,268
